In [ ]:
import re
import os
import sys
import time
import codecs
import random
import shutil
import smtplib
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as BS
from pathlib import Path
from chardet import detect
from functools import reduce
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
%config Completer.use_jedi = False

In [ ]:
def mkdirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def dir_empty(dir_path):
    return not next(os.scandir(dir_path), None)

###  爬蟲之前

<span style="color:coral">先觀察</span>

### 開始囉

#### setting header

In [ ]:
def _requests_session():
    session = requests.session()
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"
    }
    session.headers.update(headers)
    return session

In [ ]:
rs = _requests_session()

In [ ]:
url = "https://www.dramexchange.com/"

In [ ]:
res = rs.get(url)

In [ ]:
res

In [ ]:
soup = BS(res.text, "lxml")

#### 找時間

In [ ]:
# soup.find("td", {"id": "NationalDramSpotPrice_show_day"})

In [ ]:
# soup.find(id="NationalDramSpotPrice_show_day")

In [ ]:
# soup.select("#NationalDramSpotPrice_show_day")

In [ ]:
# soup.find("td", {"id": "NationalDramSpotPrice_show_day"}).parent

https://www.w3schools.com/jquery/jquery_selectors.asp

#### 取時間

https://docs.python.org/3/library/time.html#time.strftime

In [ ]:
soup.find(id="NationalDramSpotPrice_show_day").text

In [ ]:
# soup.find(id="NationalDramSpotPrice_show_day").contents[1]

In [ ]:
# soup.find(id="NationalDramSpotPrice_show_day").contents[1].contents[0]

In [ ]:
soup.find(id="NationalDramSpotPrice_show_day").contents[1].text

In [ ]:
text_time = soup.find(id="NationalDramSpotPrice_show_day").text

In [ ]:
str_time = re.findall("\w+[\. ]?\d+ \d{4}", text_time)[0]
str_time

In [ ]:
str_time.replace(".", "")

In [ ]:
time_dram = time.strptime(str_time.replace(".", ""), "%b%d %Y")
time_dram

https://stackoverflow.com/questions/12255932/what-does-the-p-in-strptime-stand-for

In [ ]:
date_dram = time.strftime("%Y-%m-%d", time_dram)
date_dram

#### 找table

In [ ]:
soup.find_all("table")

In [ ]:
soup.find("tbody", {"id": "tb_NationalDramSpotPrice"}).parent

In [ ]:
# soup.find_all("tbody", {"id": "tb_NationalDramSpotPrice"})[0].parent

In [ ]:
table_dram = soup.find("tbody", {"id": "tb_NationalDramSpotPrice"}).parent

In [ ]:
pd.read_html(table_dram.prettify(), header=None)

In [ ]:
pd.read_html(table_dram.prettify(), header=None)[0]

In [ ]:
df_dram = pd.read_html(table_dram.prettify(), header=0)[0]
df_dram

In [ ]:
df_dram_daily = df_dram[["Item", "Session Average"]]
df_dram_daily

#### Pandas

In [ ]:
cols = ["Item", date_dram]
cols

In [ ]:
df_dram_daily.columns = cols
df_dram_daily

In [ ]:
file_name = "dram-{}.csv".format(date_dram)
file_name

In [ ]:
folder_path = "./dram-daily"
folder_path

In [ ]:
mkdirs(folder_path)

In [ ]:
file_path = "{}/{}".format(folder_path, file_name)
file_path

In [ ]:
df_dram_daily.to_csv(file_path, index=False, encoding="utf-8-sig")

In [ ]:
folder_merge = "./dram-merge"
folder_merge

#### 合併之前

檢查有沒有第一天，沒有就把單天輸出當第一天

In [ ]:
merge_folder = "./dram-merge"

In [ ]:
mkdirs(merge_folder)

In [ ]:
if dir_empty(merge_folder):
    first_merge_file = "dram-merge-{}.csv".format(date_spot)
    first_merge_path = "{}/{}".format(merge_folder, first_merge_file)
    df_daily.to_csv(first_merge_path, index=False, encoding="utf-8-sig")

#### 合併新的table

https://pandas.pydata.org/docs/user_guide/merging.html

In [ ]:
os.listdir(folder_merge)[-1]

In [ ]:
list((Path(folder_merge).iterdir()))[0].name

In [ ]:
list((Path(folder_merge).iterdir()))[0].stem

In [ ]:
list((Path(folder_merge).iterdir()))[0].suffix

In [ ]:
date_dram not in list((Path(folder_merge).iterdir()))[0].stem

In [ ]:
merge_path_0 = "{}/{}".format(folder_merge, os.listdir(folder_merge)[-1])
merge_path_0

In [ ]:
df_0 = pd.read_csv(merge_path_0, encoding="utf-8")
df_0

In [ ]:
df_dram_daily

In [ ]:
df_0.append(df_dram_daily)

In [ ]:
pd.concat([df_0, df_dram_daily], axis=1)

In [ ]:
pd.merge(df_0, df_dram_daily)

In [ ]:
df_merge = pd.merge(df_0, df_dram_daily)

In [ ]:
merge_file = "dram-merge-{}.csv".format(date_dram)
merge_file

In [ ]:
merge_path = "{}/{}".format(folder_merge, merge_file)
merge_path

In [ ]:
df_merge.to_csv(merge_path, index=False, encoding="utf-8-sig")

舊的不想留

In [ ]:
a = os.listdir(folder_merge)
a

In [ ]:
str(list((Path(folder_merge).iterdir()))[0])

In [ ]:
list((Path(folder_merge).iterdir()))[0].name

In [ ]:
list(Path(folder_merge).glob("*"))

In [ ]:
b = Path(folder_merge).iterdir()
b

In [ ]:
c = [x for x in b if x.name != merge_file]
c

In [ ]:
for i in c:
    Path(i).unlink(missing_ok=True)

#### 抓表弄成function

In [ ]:
def extract_table(time_id, table_id):
    url = "https://www.dramexchange.com/"
    rs = _requests_session()
    res = rs.get(url)
    soup = BS(res.text, "lxml")
    text_time = soup.find(id=time_id).text
    str_time = re.findall("\w+[\. ]?\d+ \d{4}", text_time)[0]
    time_spot = time.strptime(str_time.replace(".", ""), "%b%d %Y")
    date_spot = time.strftime("%Y-%m-%d", time_spot)
    table_spot = soup.find("tbody", {"id": table_id}).parent
    df_spot = pd.read_html(table_spot.prettify(), header=0)[0]
    df_spot_daily = df_spot[["Item", "Session Average"]]
    cols = ["Item", date_spot]
    df_spot_daily.columns = cols
    time.sleep(random.uniform(1, 2))
    return df_spot_daily

In [ ]:
dram_id = ["NationalDramSpotPrice_show_day", "tb_NationalDramSpotPrice"]
flash_id = ["NationalFlashSpotPrice_show_day", "tb_NationalFlashSpotPrice"]

In [ ]:
element = [dram_id, flash_id]
element

In [ ]:
dfs_spot = [extract_table(x, y) for x, y in element]

In [ ]:
dfs_spot

In [ ]:
df_daily = pd.concat(dfs_spot, axis=0, join="outer")
df_daily

In [ ]:
date_spot = df_daily.columns[-1]
date_spot

In [ ]:
file_name = "dram-{}.csv".format(date_spot)
folder_path = "./dram-daily"
mkdirs(folder_path)
file_path = "{}/{}".format(folder_path, file_name)
df_daily.to_csv(file_path, index=False, encoding="utf-8-sig")

#### merge folder

檢查有沒有第一天，沒有就把單天輸出當第一天

In [ ]:
merge_folder = "./dram-merge"

In [ ]:
mkdirs(merge_folder)

In [ ]:
if dir_empty(merge_folder):
    first_merge_file = "dram-merge-{}.csv".format(date_spot)
    first_merge_path = "{}/{}".format(merge_folder, first_merge_file)
    df_daily.to_csv(first_merge_path, index=False, encoding="utf-8-sig")

#### 今天合併以前

In [ ]:
last_merge_path = list((Path(merge_folder).iterdir()))[-1]

In [ ]:
df_merged = pd.read_csv(last_merge_path, encoding="utf-8-sig")

In [ ]:
df_merge = df_merged.merge(df_daily, how="outer", on="Item")
df_merge

In [ ]:
df_merge.columns = df_merge.columns.str.replace("_y", "")

In [ ]:
df_merge

In [ ]:
df_merge = df_merge.drop(list(df_merge.filter(regex = "_")), axis=1, inplace=False)

In [ ]:
df_merge

In [ ]:
merge_filename = "dram-merge-{}.csv".format(date_spot)
merge_path = "{}/{}".format(merge_folder, merge_filename)

In [ ]:
df_merge.to_csv(merge_path, index=False, encoding="utf-8-sig")

#### 統整起來

In [ ]:
import re
import os
import sys
import time
import codecs
import random
import shutil
import smtplib
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as BS
from pathlib import Path
from chardet import detect
from functools import reduce
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta